In [7]:
import pandas as pd
import requests
import json
from pandas import json_normalize


In [8]:
def get_GD_data(url):
  url = url
  #buscando dados da API#######
  #Buscando com várias requisições
  df_aneel = pd.DataFrame()
  #dados = pd.DataFrame()
  
  offset = 0
  while True:
        url_base = url
        url_x = url_base + '&offset=' + str(offset)
        #fazendo o request da web page e retornando um objeto json
        response = requests.get(url_x).json()
        r = json.dumps(response)
        #converse a str json para py dict 
        loaded_r = json.loads(r)
        #print(loaded_r)
        total_data_rows = loaded_r['result']['total']
        #busca a tabela de dados desejada
        dados = json_normalize(loaded_r['result']['records'])
        #dados = pd.DataFrame(loaded_r['result']['records'])
        df_aneel = pd.concat([df_aneel, dados])
        offset += 32000
        #print(offset, total_data_rows)
        if offset >= total_data_rows:
          #print('estou dentro do if')

          break
        
  
  
  
  
  
  
  
  #filtra as colunas
  dados_aneel = df_aneel.loc[:, ['DatGeracaoConjuntoDados',
                              'SigAgente', 'DscClasseConsumo', 
                              'DscClasseConsumo', 'CodSubGrupoTarifario', 
                              'DscSubGrupoTarifario', 'SigUF', 'NomRegiao', 
                              'NomMunicipio', 'SigTipoConsumidor', 'NumCPFCNPJ',
                              'SigModalidadeEmpreendimento', 'SigTipoGeracao', 
                              'DscPorte', 'NumCoordNEmpreendimento', 'NumCoordEEmpreendimento',
                              'MdaPotenciaInstaladaKW']]
  #buscas as linhas referentes a geração no RN
  #dados_aneel = dados_aneel.loc[dados_aneel.SigUF == 'RN', :]
  #remove linhas sem a informação da UF
  dados_aneel = dados_aneel.dropna(subset=['SigUF'])
  #filtra dados de apenas usinas fotovoltaicas 
  dados_aneel = dados_aneel.loc[dados_aneel.SigTipoGeracao == 'UFV', :]
  #cria novas colunas referentes ao mês e ano (pode ajudar na fase de visualização gráfica)
  col_data = pd.DatetimeIndex(dados_aneel['DatGeracaoConjuntoDados'])
  dados_aneel['Ano'] = col_data.year
  dados_aneel['Mes'] = col_data.month_name()
  #converte os valores objeto --> float
  dados_aneel['MdaPotenciaInstaladaKW'] = dados_aneel['MdaPotenciaInstaladaKW'].str.replace(',', '.').astype(float)

  return dados_aneel.to_csv('/content/drive/MyDrive/Data_analsys/Dados_ANEEL.csv', index=False)







In [9]:
url = 'https://dadosabertos.aneel.gov.br/api/3/action/datastore_search?resource_id=b1bd71e7-d0ad-4214-9053-cbd58e9564a7&q=RN&limit=32000'
get_GD_data(url)
